In [1]:
import cvxpy as cp
import numpy as np

import torch
import torch.nn as nn
from model import TwolayerNet, MLP, AbstainNet
from torch.utils.data import DataLoader
from utils import load_data

import os
import time
from options import args_parser

import matplotlib.pyplot as plt

In [2]:
class namespace:
    def __init__(self):
        self.batch_size=128
        self.device='cuda:1'
        self.epoch=30000
        self.hidden_dim=300
        self.input_dim=108
        self.load_from_disk=True
        self.lr=0.0005
        self.seed=1
        self.fairness_notion="EO"
        self.attribute="sex"
        self.type_of_abstain="prob"

args = namespace()

print(args)

args.model_path = "../data/OptimalClassifier"
args.data_path = "../data/adult"

train_data, test_data = load_data(args)

In [3]:
def load_optimal(args):
    model = MLP(args)

    checkpoint = torch.load(os.path.join(args.model_path, "model_state.pth"), map_location="cpu")
    model.load_state_dict(checkpoint)

    model.to("cpu")
    model.eval()

    return model
def load_abstain(args):
    model = AbstainNet(args)
    
    model_file = os.path.join("../data/AbstainClassifier", "abstain_classifier_" + \
                            args.type_of_abstain+"_" + \
                            args.fairness_notion + "_" + \
                            args.attribute + ".pth")
    
    checkpoint = torch.load(model_file, map_location="cpu")
    model.load_state_dict(checkpoint)
    
    model.to("cpu")
    model.eval()
    
    return model

def predict_abstain(model, data, pred_labels):
    X = data.X.numpy()
    X = np.column_stack((X, pred_labels))
    log_probs = model(torch.from_numpy(X).to(torch.float))
    return log_probs.detach().cpu().numpy().flatten()

def predict(model, data):
    log_probs = model(data.X)
    return ((log_probs[:, 1] >= 0.5) * 1).detach().cpu().numpy(), log_probs[:, 1].detach().cpu().numpy()

def formatted(numbers):
    return [f"{num*100:.2f}%" for num in numbers]

In [4]:
data = train_data
optimal_classifier = load_optimal(args)
pred_labels, pred_probs = predict(optimal_classifier, data)
# np.save("../data/adult/optimal_labels.npy", pred_labels)
pred_labels_test, _ = predict(optimal_classifier, test_data)
# np.save("../data/adult/test_optimal_labels.npy", pred_labels_test)

**LP Optimazation**

In [5]:
delta = 0.3
epsilon = 0.03
num_samples = 2000

In [8]:
X = data.X[:num_samples].numpy()
y = data.y[:num_samples].numpy().astype(int)
pred_labels = pred_labels[:num_samples]

n = X.shape[0]
g2_indices = np.where(X[:, 64] == 1)[0]
g1_indices = np.where(X[:, 65] == 1)[0]

g1_v = X[:, 65]
g2_v = X[:, 64]

g1_num = g1_indices.shape[0]
g2_num = g2_indices.shape[0]
total_error = pred_labels != y
g1_error = (pred_labels != y)[g1_indices]
g2_error = (pred_labels != y)[g2_indices]
g1_error_rate = np.sum(g1_error) / g1_num
g2_error_rate = np.sum(g2_error) / g2_num
total_error_rate = np.sum(total_error) / n

e1 = (pred_labels * g1_v * y).reshape(-1, 1)
t1 = (g1_v * y).reshape(-1, 1)

e2 = (pred_labels * g2_v * y).reshape(-1, 1)
t2 = (g2_v * y).reshape(-1, 1)

A_EO = np.zeros([e1.shape[0], e1.shape[0]])
B_EO = np.zeros([e1.shape[0], e1.shape[0]])
A_EO = - e1 * t2.T + e2 * t1.T + epsilon * t1 * t2.T
B_EO = - e2 * t1.T + e1 * t2.T + epsilon * t1 * t2.T

In [13]:
w = cp.Variable(n)

objective = cp.Minimize(cp.sum(cp.multiply(total_error, w)))

constraints = [
    w <= 1,
    w >= 0,
    
    cp.quad_form(w, A_EO) <= 0,
    cp.quad_form(w, B_EO) <= 0,
    
    cp.sum(w[g1_indices]) >= (1 - delta) * g1_num,
    cp.sum(w[g2_indices]) >= (1 - delta) * g2_num,
    cp.sum(cp.multiply(w[g1_indices], g1_error)) <= g1_error_rate * cp.sum(w[g1_indices]),
    cp.sum(cp.multiply(w[g2_indices], g2_error)) <= g2_error_rate * cp.sum(w[g2_indices]),
]

ValueError: Quadratic form matrices must be symmetric/Hermitian.

In [14]:
prob = cp.Problem(objective, constraints)
result = prob.solve()
wn = w.value

NameError: name 'constraints' is not defined